## Shopify Intern - Data Science

<font color='red'> Question 1: Given some sample data, write a program to answer the following: click here to access the required data set

<font color='red'> On Shopify, we have exactly 100 sneaker shops, and each of these shops sells only one model of shoe. We want to do some analysis of the average order value (AOV). When we look at orders data over a 30 day window, we naively calculate an AOV of $3145.13. Given that we know these shops are selling sneakers, a relatively affordable item, something seems wrong with our analysis. 

<font color='red'> 1. Think about what could be going wrong with our calculation. Think about a better way to evaluate this data. <br>
<font color='red'> 2. What metric would you report for this dataset?<br>
<font color='red'> 3. What is its value? </font>


### 1) Why is AOV high for affordable item like sneaker? Think about what could be going wrong with our calculation. Think about a better way to evaluate this data. 

I quickly scrolled down the data and found some order_amount were extremely high. 
Few people bought 2000 pairs of sneakers which has affected the AOV value. Majority people bought 1 or 2 pairs but few people bought so many pairs of sneakers.

So, let us do some data analysis to see what is happening!!


In [2]:
import pandas as pd

In [4]:
data = pd.read_csv('2019 Winter Data Science Intern Challenge Data Set - Sheet1.csv')
data.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017-03-13 12:36:56
1,2,92,925,90,1,cash,2017-03-03 17:38:52
2,3,44,861,144,1,cash,2017-03-14 4:23:56
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11


In [24]:
#Currently calculated AOV
data['order_amount'].sum()/len(data)

3145.128

Now that we know how high AOV was calculated, let us explore the data and see what is going wrong.

In [5]:
data.describe()

,order_id,shop_id,user_id,order_amount,total_items
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000
mean,2500.500000,50.078800,849.092400,3145.128000,8.78720
std,1443.520003,29.006118,87.798982,41282.539349,116.32032
min,1.000000,1.000000,607.000000,90.000000,1.00000
25%,1250.750000,24.000000,775.000000,163.000000,1.00000
50%,2500.500000,50.000000,849.000000,284.000000,2.00000
75%,3750.250000,75.000000,925.000000,390.000000,3.00000
max,5000.000000,100.000000,999.000000,704000.000000,2000.00000


- Standard deviation of order_amount is 41282.53 (very high!!)
- Standard deviation of total_items is 116.32 (high)
- Max of order_amount is 70,4000 (very high!!)
- Max of total_items is 2000 (very high)

<font color='blue'> High AOV could be because of outliers. Let us explore the data to find out. 

In [44]:
# Let us look at how the order amounts vary
data.groupby(['order_amount']).size().reset_index(name='counts').sort_values(['order_amount'], ascending=False).head(10)


,order_amount,counts
257,704000,17
256,154350,1
255,102900,1
254,77175,9
253,51450,16
252,25725,19
251,1760,1
250,1408,2
249,1086,1
248,1064,1


An order worth a lot of money were ordred multiple times. (704000 worth 17 times??, 77175 worth 9 times??)
Something is seriously wrong!!

In [42]:
# Let us see how many sneakers people are buying at a time
data.groupby(['total_items']).size().reset_index(name='counts').sort_values(['total_items'], ascending=False).head(10)


,total_items,counts
7,2000,17
6,8,1
5,6,9
4,5,77
3,4,293
2,3,941
1,2,1832
0,1,1830


17 times, 2000 sneakers were ordered. That's a lot. We should look at those orders 

In [51]:
#Let us look at these orders
data.loc[(data['total_items'] == 2000)].sort_values(['created_at'], ascending=False)

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
2835,2836,42,607,704000,2000,credit_card,2017-03-28 4:00:00
4056,4057,42,607,704000,2000,credit_card,2017-03-28 4:00:00
2969,2970,42,607,704000,2000,credit_card,2017-03-28 4:00:00
4882,4883,42,607,704000,2000,credit_card,2017-03-25 4:00:00
1104,1105,42,607,704000,2000,credit_card,2017-03-24 4:00:00
3332,3333,42,607,704000,2000,credit_card,2017-03-24 4:00:00
4868,4869,42,607,704000,2000,credit_card,2017-03-22 4:00:00
1562,1563,42,607,704000,2000,credit_card,2017-03-19 4:00:00
1602,1603,42,607,704000,2000,credit_card,2017-03-17 4:00:00
1362,1363,42,607,704000,2000,credit_card,2017-03-15 4:00:00


User 42, is buying 2000 sneakers on multiple days exactly at 4 AM and sometimes multiple times at exactly 4 AM.
There is either a glitch in system that is logging wrong data, or the user has automated his purchase. 
But this is a lot of money to spend on sneakers.

In [60]:
#Order amount greater 2000. I picked 2000 because of the sudden jump from 1760 to 25725
data.loc[(data['order_amount'] >= 2000)]

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
15,16,42,607,704000,2000,credit_card,2017-03-07 4:00:00
60,61,42,607,704000,2000,credit_card,2017-03-04 4:00:00
160,161,78,990,25725,1,credit_card,2017-03-12 5:56:57
490,491,78,936,51450,2,debit,2017-03-26 17:08:19
493,494,78,983,51450,2,cash,2017-03-16 21:39:35
...,...,...,...,...,...,...,...
4646,4647,42,607,704000,2000,credit_card,2017-03-02 4:00:00
4715,4716,78,818,77175,3,debit,2017-03-05 5:10:44
4868,4869,42,607,704000,2000,credit_card,2017-03-22 4:00:00
4882,4883,42,607,704000,2000,credit_card,2017-03-25 4:00:00


Some users bought 2 pair of sneakers for 51,450 dollars and paid cash!!
This looks like a fraud transaction. This is definitely worth further investigation!


<font color ='blue'>On exploring the data we found - 
* <font color ='blue'> Multiple orders worth huge amounts were performed by same user at same time 4AM, multiple times a month.
* <font color ='blue'> Same user (42), sometimes paid 51,450 in cash for 2 shoes. This user could be performing some fraudulant transactions!! <br>
All these transactions are skewing our AOV value
    

<font color ='blue'> Better way to calculate this data would be to identify and remove orders that look fraudulant, and then calculate AOV

In [65]:
data_new = data.drop(data[(data.total_items == 2000) | (data.order_amount >=2000)].index)
data_new

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017-03-13 12:36:56
1,2,92,925,90,1,cash,2017-03-03 17:38:52
2,3,44,861,144,1,cash,2017-03-14 4:23:56
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11
...,...,...,...,...,...,...,...
4995,4996,73,993,330,2,debit,2017-03-30 13:47:17
4996,4997,48,789,234,2,cash,2017-03-16 20:36:16
4997,4998,56,867,351,3,cash,2017-03-19 5:42:42
4998,4999,60,825,354,2,credit_card,2017-03-16 14:51:18


In [66]:
#Let us now look at the stats for data_new
data_new.describe()

,order_id,shop_id,user_id,order_amount,total_items
count,4937.000000,4937.000000,4937.000000,4937.000000,4937.000000
mean,2499.551347,49.846465,849.752279,302.580514,1.994734
std,1444.069407,29.061131,86.840313,160.804912,0.982821
min,1.000000,1.000000,700.000000,90.000000,1.000000
25%,1248.000000,24.000000,775.000000,163.000000,1.000000
50%,2497.000000,50.000000,850.000000,284.000000,2.000000
75%,3751.000000,74.000000,925.000000,387.000000,3.000000
max,5000.000000,100.000000,999.000000,1760.000000,8.000000


We see the AOV is 302.58 


### 2) What metric would you report for this dataset?

<font color='blue'> We see a lot of orders with lower order_amount and order_amount value ranges from 90 to 1760. So, instead of calculating mean, <b>median</b> would be a better metric

### 3) What is its value?

In [76]:
data_new['order_amount'].median()

284.0

<font color='blue'>Median value is 284.0 </font>

<font color='red'> Question 2: For this question you’ll need to use SQL. Follow this link to access the data set required for the challenge. Please use queries to answer the following questions. Paste your queries along with your final numerical answers below.<br>

<font color='red'> 1. How many orders were shipped by Speedy Express in total? <br>
<font color='red'> 2. What is the last name of the employee with the most orders? <br>
<font color='red'> 3. What product was ordered the most by customers in Germany?



### 1. How many orders were shipped by Speedy Express in total?

In [ ]:
SELECT count(*) as order_count 
FROM Orders as o JOIN Shippers as s
ON o.ShipperID = s.ShipperID
WHERE s.ShipperName = "Speedy Express"

<font color='blue'> 54

### 2. What is the last name of the employee with the most orders?

In [ ]:
SELECT e.LastName, count(o.OrderID) as order_count
FROM Employees as e JOIN Orders as o
ON e.EmployeeID = o.EmployeeID
GROUP BY e.LastName
ORDER BY order_count desc
LIMIT 1

<font color = 'blue'> Peacock

### 3. What product was ordered the most by customers in Germany?

In [ ]:
SELECT p.ProductName, SUM(od.Quantity) as product_quantity
FROM Products as p 
JOIN OrderDetails as od
    ON p.ProductID = od.ProductID
JOIN Orders as o
    ON od.OrderID = o.OrderID
JOIN Customers as c
    ON o.CustomerID = c.CustomerID
WHERE c.Country = 'Germany'
GROUP BY p.ProductName
ORDER BY product_quantity desc
LIMIT 1

<font color='blue'> Boston Crab Meat